# PQ-FSR Examples

This notebook demonstrates the usage of the Post-Quantum Forward-Secret Ratchet (PQ-FSR) reference implementation.


In [ ]:
from pqfsr import RatchetSession

print("PQ-FSR imported successfully")


## Basic Usage

The basic workflow involves:
1. Creating initiator and responder sessions
2. Performing a handshake
3. Encrypting and decrypting messages


In [ ]:
# Create sessions
alice = RatchetSession.create_initiator(semantic_hint=b"alice", max_skip=32)
bob = RatchetSession.create_responder(semantic_hint=b"bob", max_skip=32)

# Perform handshake
request = alice.create_handshake_request()
print(f"Handshake request created: {len(request)} fields")

response = bob.accept_handshake(request)
print(f"Handshake response created: {len(response)} fields")

alice.finalize_handshake(response)
print("Handshake completed successfully!")


In [ ]:
# Encrypt and decrypt a message
message = b"Hello, quantum world!"
packet = alice.encrypt(message)
print(f"Encrypted packet size: {len(str(packet))} bytes")

plaintext = bob.decrypt(packet)
print(f"Decrypted message: {plaintext.decode()}")
assert plaintext == message
print("✓ Message roundtrip successful!")


## Bidirectional Communication

Messages can be sent in both directions after the handshake.


In [ ]:
# Alice sends a message
alice_msg = b"Hi Bob!"
alice_packet = alice.encrypt(alice_msg)
bob_received = bob.decrypt(alice_packet)
print(f"Bob received: {bob_received.decode()}")

# Bob replies
bob_msg = b"Hi Alice!"
bob_packet = bob.encrypt(bob_msg)
alice_received = alice.decrypt(bob_packet)
print(f"Alice received: {alice_received.decode()}")


## State Serialization

Session state can be exported and restored, useful for persistence across application restarts.


In [ ]:
# Export Bob's state
bob_state = bob.export_state()
print(f"Exported state size: {len(bob_state)} bytes")

# Restore Bob's state in a new session
bob_restored = RatchetSession.from_serialized(bob_state)

# Verify it works
test_message = b"State restored successfully!"
packet = alice.encrypt(test_message)
plaintext = bob_restored.decrypt(packet)
print(f"Restored session decrypted: {plaintext.decode()}")
assert plaintext == test_message
print("✓ State serialization successful!")


## Post-Compromise Recovery

PQ-FSR provides post-compromise security: if an attacker steals session state, they cannot decrypt future messages after the next honest message exchange.


In [ ]:
# Simulate a compromise scenario
alice_pc = RatchetSession.create_initiator(semantic_hint=b"alice", max_skip=32)
bob_pc = RatchetSession.create_responder(semantic_hint=b"bob", max_skip=32)

req = alice_pc.create_handshake_request()
resp = bob_pc.accept_handshake(req)
alice_pc.finalize_handshake(resp)

# Send a message
packet1 = alice_pc.encrypt(b"before-compromise")
bob_pc.decrypt(packet1)

# Attacker steals Alice's state
compromised_state = alice_pc.export_state()
compromised = RatchetSession.from_serialized(compromised_state)

# Alice continues and sends another message
packet2 = alice_pc.encrypt(b"after-compromise")
bob_pc.decrypt(packet2)  # Bob can still decrypt

# Attacker tries to decrypt the new message
try:
    compromised.decrypt(packet2)
    print("✗ Security breach! Compromised state decrypted future message")
except ValueError as e:
    print(f"✓ Post-compromise security: {e}")
    print("Compromised state cannot decrypt future messages")


## Error Handling

The library provides clear error messages for invalid operations.


In [ ]:
# Try to encrypt before handshake
alice_err = RatchetSession.create_initiator(semantic_hint=b"alice")
try:
    alice_err.encrypt(b"too early")
except ValueError as e:
    print(f"Expected error: {e}")

# Try to decrypt a tampered packet
alice_tamper = RatchetSession.create_initiator(semantic_hint=b"alice")
bob_tamper = RatchetSession.create_responder(semantic_hint=b"bob")

req = alice_tamper.create_handshake_request()
resp = bob_tamper.accept_handshake(req)
alice_tamper.finalize_handshake(resp)

packet = alice_tamper.encrypt(b"original")
tampered = packet.copy()
tampered["ciphertext"] = b"tampered" + b"\x00" * 100

try:
    bob_tamper.decrypt(tampered)
except ValueError as e:
    print(f"Tampering detected: {e}")


## Custom KEM Usage

You can provide a custom KEM implementation for testing or integration with real post-quantum KEMs.


In [ ]:
from pqfsr import InMemoryKEM
import hashlib

# Create deterministic KEMs for testing
def make_rng(seed):
    counter = 0
    def rng(n):
        nonlocal counter
        buffer = bytearray()
        while len(buffer) < n:
            block = hashlib.sha256(seed + counter.to_bytes(4, "big")).digest()
            buffer.extend(block)
            counter += 1
        return bytes(buffer[:n])
    return rng

kem_a = InMemoryKEM(make_rng(b"kem-seed-a"))
kem_b = InMemoryKEM(make_rng(b"kem-seed-b"))

alice_custom = RatchetSession.create_initiator(
    semantic_hint=b"alice",
    kem=kem_a
)
bob_custom = RatchetSession.create_responder(
    semantic_hint=b"bob",
    kem=kem_b
)

req = alice_custom.create_handshake_request()
resp = bob_custom.accept_handshake(req)
alice_custom.finalize_handshake(resp)

packet = alice_custom.encrypt(b"custom KEM test")
plaintext = bob_custom.decrypt(packet)
print(f"Custom KEM works: {plaintext.decode()}")


## Associated Data

You can include associated data (context) that is authenticated but not encrypted.


In [ ]:
alice_ad = RatchetSession.create_initiator(semantic_hint=b"alice")
bob_ad = RatchetSession.create_responder(semantic_hint=b"bob")

req = alice_ad.create_handshake_request()
resp = bob_ad.accept_handshake(req)
alice_ad.finalize_handshake(resp)

# Encrypt with associated data
associated_data = b"message-context: user-id-12345"
packet = alice_ad.encrypt(b"secret message", associated_data=associated_data)

# Decrypt with same associated data
plaintext = bob_ad.decrypt(packet, associated_data=associated_data)
print(f"Decrypted with AD: {plaintext.decode()}")

# Wrong associated data fails
try:
    bob_ad.decrypt(packet, associated_data=b"wrong-context")
except ValueError as e:
    print(f"Wrong AD rejected: {e}")


## Multiple Messages

The ratchet handles multiple messages efficiently, with forward secrecy maintained for each message.


In [ ]:
alice_multi = RatchetSession.create_initiator(semantic_hint=b"alice")
bob_multi = RatchetSession.create_responder(semantic_hint=b"bob")

req = alice_multi.create_handshake_request()
resp = bob_multi.accept_handshake(req)
alice_multi.finalize_handshake(resp)

# Send multiple messages
messages = [b"message-1", b"message-2", b"message-3", b"message-4"]
packets = [alice_multi.encrypt(msg) for msg in messages]

# Decrypt all
for i, packet in enumerate(packets):
    plaintext = bob_multi.decrypt(packet)
    print(f"Message {i+1}: {plaintext.decode()}")
    assert plaintext == messages[i]

print("✓ All messages decrypted successfully!")
